In [ ]:
import librosa
from google.colab import drive
import os
import numpy as np
from pydub import AudioSegment, effects
import noisereduce as nr
import matplotlib.pyplot as plt
from librosa import display   
import IPython.display as ipd
import numpy as np
import time
import os
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import optimizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint



drive.mount('/content/drive/', force_remount=True)
data , sampling_rate =librosa.load( '/content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_01/03-01-03-02-01-01-01.wav')
#-------------------------------------------------------------------------------------
path = '/content/drive/My Drive/speech-emotion-recognition-ravdess-data'
lst = []

start_time = time.time()

for subdir, dirs, files in os.walk(path):
  for file in files:
      try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(file[7:8]) - 1 
        arr = mfccs, file
        lst.append(arr)
      # If the file is not valid, skip it
      except ValueError:
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))
#--------------------------------------------------------------------------------------
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)
x_traincnn.shape, x_testcnn.shape


model = Sequential()

model.add(Conv1D(64, 5,padding='same',
                 input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(256, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)
model.summary()


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
cnnhistory=model.fit(x_traincnn, y_train, batch_size=10, epochs=200, validation_data=(x_testcnn, y_test))



97/97 [==============================] - 1s 13ms/step - loss: 1.3065 - accuracy: 0.5187 - val_loss: 1.3918 - val_accuracy: 0.4832
Epoch 132/200
97/97 [==============================] - 1s 13ms/step - loss: 1.2573 - accuracy: 0.5332 - val_loss: 1.4041 - val_accuracy: 0.4811
Epoch 133/200
97/97 [==============================] - 1s 12ms/step - loss: 1.2841 - accuracy: 0.5187 - val_loss: 1.3976 - val_accuracy: 0.4790
Epoch 134/200
97/97 [==============================] - 1s 12ms/step - loss: 1.2397 - accuracy: 0.5488 - val_loss: 1.3938 - val_accuracy: 0.4811
Epoch 135/200
97/97 [==============================] - 1s 12ms/step - loss: 1.2195 - accuracy: 0.5384 - val_loss: 1.4071 - val_accuracy: 0.4811
Epoch 136/200
97/97 [==============================] - 1s 13ms/step - loss: 1.2355 - accuracy: 0.5239 - val_loss: 1.3893 - val_accuracy: 0.5105
Epoch 137/200
97/97 [==============================] - 1s 13ms/step - loss: 1.2338 - accuracy: 0.5332 - val_loss: 1.4276 - val_accuracy: 0.4706
Epoch 